In [1]:
import numpy as np
np.random.seed(1)

from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Input
from keras.preprocessing.text import one_hot
#from keras.preprocessing.sequence import pad_sequences
#from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder

In [2]:
# define documents

docs=['king is man',
'a king marrys queen',
'a queen marrys king',
'some unrelated workds'
'queen is woman']
docs

['king is man',
 'a king marrys queen',
 'a queen marrys king',
 'some unrelated workdsqueen is woman']

In [3]:
vocab_size=50

In [4]:
oh = np.array(one_hot('king queen man woman unrelated', vocab_size)).reshape(-1,1)
oh

array([[16],
       [17],
       [41],
       [39],
       [16]])

In [5]:
enc=OneHotEncoder()
enc.fit(np.array(range(50)).reshape(-1,1))
oh_enc=enc.transform(oh).toarray()
oh_enc

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [6]:
encoded_docs=[one_hot(d, vocab_size) for d in docs]
encoded_docs

[[16, 47, 41], [9, 16, 48, 17], [9, 17, 48, 16], [43, 16, 41, 47, 39]]

In [7]:
max_length=24
padded_docs=pad_sequences(encoded_docs, maxlen=max_length, padding='post')
padded_docs

array([[16, 47, 41,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 9, 16, 48, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 9, 17, 48, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [43, 16, 41, 47, 39,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0]])

In [8]:
tuples = np.empty((0, 2))
for padded_doc in padded_docs:
    length = len(padded_doc)
    for i in range(length):
        if padded_doc[i] != 0:
            if i<length-1 & padded_doc[i+1] != 0:
                tuples = np.append(tuples, [[padded_doc[i],padded_doc[i+1]]], axis=0) 
                if i<length-2 & padded_doc[i+2] != 0:
                    tuples = np.append(tuples, [[padded_doc[i],padded_doc[i+2]]], axis=0) 
            if i > 0:
                tuples = np.append(tuples, [[padded_doc[i],padded_doc[i-1]]], axis=0) 
                if i > 1:
                    tuples = np.append(tuples, [[padded_doc[i],padded_doc[i-2]]], axis=0) 

    
print (tuples.shape)
tuples

(35, 2)


array([[16., 47.],
       [47., 16.],
       [41., 47.],
       [41., 16.],
       [ 9., 16.],
       [ 9., 48.],
       [16., 48.],
       [16., 17.],
       [16.,  9.],
       [48., 17.],
       [48., 16.],
       [48.,  9.],
       [17., 48.],
       [17., 16.],
       [ 9., 17.],
       [ 9., 48.],
       [17., 48.],
       [17., 16.],
       [17.,  9.],
       [48., 16.],
       [48., 17.],
       [48.,  9.],
       [16., 48.],
       [16., 17.],
       [43., 16.],
       [16., 43.],
       [41., 47.],
       [41., 39.],
       [41., 16.],
       [41., 43.],
       [47., 39.],
       [47., 41.],
       [47., 16.],
       [39., 47.],
       [39., 41.]])

In [9]:
onehotlabels_x=enc.transform(tuples[:,0].reshape(-1,1)).toarray()

print(onehotlabels_x.shape)

print("---------")

print(onehotlabels_x)

print("---------")

print(onehotlabels_x[0])

(35, 50)
---------
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
---------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [10]:
onehotlabels_y=enc.transform(tuples[:,1].reshape(-1,1)).toarray()

print(onehotlabels_y.shape)

print("---------")

print(onehotlabels_y)

print("---------")

print(onehotlabels_y[0])

(35, 50)
---------
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
---------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0.]


In [11]:
model=Sequential()

input=Dense(50, input_shape=(50,), activation='relu')
model.add(input)

bottleneck=Dense(2, activation='relu')
model.add(bottleneck)
model.add(Dense(50, activation="softmax"))

# compile the model
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['acc'])

# summarize the model
print(model.summary())

# fit the model
model.fit(onehotlabels_x, onehotlabels_y, epochs=500, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 2)                 102       
                                                                 
 dense_2 (Dense)             (None, 50)                150       
                                                                 
Total params: 2,802
Trainable params: 2,802
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
2/2 [==============================] - 1s 24ms/step - loss: 0.6956 - acc: 0.0000e+00
Epoch 2/500
2/2 [==============================] - 0s 7ms/step - loss: 0.6941 - acc: 0.0000e+00
Epoch 3/500
2/2 [==============================] - 0s 6ms/step - loss: 0.6927 - acc: 0.0000e+00
Epoch 4/500
2/2 [=======

In [12]:
model = Sequential()
model.add(input)
model.add(bottleneck)
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['acc'])
test = model.predict(oh_enc)
print (test)

C:\Users\mdana\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


1/1 [==============================] - 0s 161ms/step
[[ 9.848846 10.518787]
 [10.024619 11.55977 ]
 [ 9.604834 10.537645]
 [ 9.029195  9.425842]
 [ 9.848846 10.518787]]


In [13]:
model = Sequential()
model.add(Embedding(1000, 2, input_length=5))
model.compile('rmsprop', 'mse')

input_array = np.random.randint(1000, size=(12, 5)) # 12 documents, 5 words per document
input_array

array([[ 37, 235, 908,  72, 767],
       [905, 715, 645, 847, 960],
       [144, 129, 972, 583, 749],
       [508, 390, 281, 178, 276],
       [254, 357, 914, 468, 907],
       [252, 490, 668, 925, 398],
       [562, 580, 215, 983, 753],
       [503, 478, 864,  86, 141],
       [393,   7, 319, 829, 534],
       [313, 513, 896, 316, 209],
       [264, 728, 653, 627, 431],
       [633, 456, 542,  71, 387]])

In [14]:
output_array = model.predict(input_array)
print (output_array.shape)
print (output_array)

1/1 [==============================] - 0s 76ms/step
(12, 5, 2)
[[[ 0.02442168 -0.02432264]
  [ 0.04766626  0.02950082]
  [ 0.03154724 -0.01708014]
  [ 0.03204086  0.03390792]
  [-0.00847467  0.00453707]]

 [[-0.03099384 -0.00909394]
  [-0.01738935  0.00497639]
  [ 0.04181557  0.04606763]
  [ 0.02008449 -0.03135581]
  [ 0.03397456  0.01948765]]

 [[-0.04170153  0.03683471]
  [ 0.0283894  -0.0407846 ]
  [-0.03797469 -0.00201968]
  [-0.02053916  0.01348615]
  [-0.04319121  0.04331007]]

 [[ 0.03633369 -0.03867989]
  [ 0.03969816 -0.01654539]
  [ 0.02561254 -0.02525103]
  [-0.00819603 -0.0238373 ]
  [-0.04287419  0.02201256]]

 [[-0.01593671  0.00505142]
  [-0.04173484  0.01656948]
  [ 0.01102317 -0.03902356]
  [ 0.02267789  0.03818962]
  [ 0.00818421 -0.00604444]]

 [[-0.01178833  0.03663734]
  [ 0.03338727  0.00806184]
  [ 0.04900718 -0.038922  ]
  [ 0.0027676  -0.01677153]
  [-0.02796398  0.01334813]]

 [[ 0.04055388  0.04241124]
  [ 0.02665753 -0.04414532]
  [ 0.03553656 -0.03911716]
 

In [15]:
# define documents
docs = ['Well done!',
'Good work',
'Great effort',
'nice work',
'Excellent!',
'Really Weak',
'Poor effort!',
'not good',
'poor work',
'Could have done better.']
# define class labels
labels = [1,1,1,1,1,0,0,0,0,0]

In [16]:
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[49, 25], [35, 15], [5, 3], [28, 15], [10], [47, 39], [11, 3], [17, 35], [11, 15], [17, 5, 25, 9]]


In [17]:
# pad documents to a max length of 4 words
max_length = 4
#padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
padded_docs = np.array(pad_sequences(encoded_docs, maxlen=max_length, padding='post'))
labels = np.array([1,1,1,1,1,0,0,0,0,0])
print(padded_docs)

[[49 25  0  0]
 [35 15  0  0]
 [ 5  3  0  0]
 [28 15  0  0]
 [10  0  0  0]
 [47 39  0  0]
 [11  3  0  0]
 [17 35  0  0]
 [11 15  0  0]
 [17  5 25  9]]


In [18]:
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 8)              400       
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
import numpy as np
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 89.999998


In [20]:
import numpy as np
np.hstack((model.predict(padded_docs),np.array(labels).reshape(10,1)))

1/1 [==============================] - 0s 95ms/step


array([[0.55922347, 1.        ],
       [0.56478161, 1.        ],
       [0.53344101, 1.        ],
       [0.55677027, 1.        ],
       [0.54427272, 1.        ],
       [0.48612186, 0.        ],
       [0.47604659, 0.        ],
       [0.48857781, 0.        ],
       [0.5077129 , 0.        ],
       [0.39910132, 0.        ]])